# SciPy統計テスト チュートリアル Notebook

`README.md` の内容を Jupyter 上で再生し、各サンプルのソースと出力をまとめて確認できます。


## 環境セットアップ
以下のコマンドで uv ベースの仮想環境を用意してから実行してください。
```bash
uv venv .venv
source .venv/bin/activate
uv pip install numpy scipy
```


## 1. 一標本 t 検定
- **対象スクリプト**: `examples/t_test_one_sample.py`
- **目的**: 平均時短が 3.5 分を超えるかどうかを一標本 t 検定で確認します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/t_test_one_sample.py`

### コマンド/オプションのポイント
- `--with numpy` でサンプル配列を即時計算環境へ追加
- `--with scipy` で `stats.ttest_1samp` を提供
- `examples/t_test_one_sample.py` を別ファイルに変えると他シナリオも再利用可能

### スクリプト確認と実行


In [ ]:
"""One-sample t-test comparing sample mean to a known population mean."""
from scipy import stats
import numpy as np

# --------------------------------------------------------------
# このスクリプトでは、単一の標本平均が事前に主張されている母平均（3.5 分）と
# 有意に異なるかどうかを一標本 t 検定で確認します。
# 帰無仮説: 新ワークフローの平均時短は 3.5 分である。
# 対立仮説: 新ワークフローの平均時短は 3.5 分を超える（左右両側で検定）。
# --------------------------------------------------------------

# 12 人の従業員が新ワークフローで節約した時間（分）。固定値にして再現性を確保。
sample = np.array([4.1, 3.7, 4.4, 5.2, 3.9, 4.0, 4.6, 3.8, 4.3, 4.1, 4.7, 3.6])
# 比較対象となる母平均。業務要件として「3.5 分の時短が必要」と仮定。
population_mean = 3.5

# stats.ttest_1samp で t 値と p 値を取得。デフォルトで両側検定になる点に注意。
statistic, p_value = stats.ttest_1samp(sample, population_mean)

print(f"t-statistic: {statistic:.3f}")
print(f"p-value: {p_value:.4f}")
# 5% を閾値にして、母平均との差が統計的に有意かどうかを判定。
if p_value < 0.05:
    print("Reject the null hypothesis: the workflow saves more than 3.5 minutes on average.")
else:
    print("Fail to reject the null hypothesis: insufficient evidence that savings exceed 3.5 minutes.")

## 2. Welch の 2 標本 t 検定
- **対象スクリプト**: `examples/t_test_independent.py`
- **目的**: 旧/新オンボーディングの満足度平均差を Welch t 検定で比較します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/t_test_independent.py`

### コマンド/オプションのポイント
- `equal_var=False` で分散が異なる前提でも対応
- `--with numpy` で 2 群の配列演算を支援
- `--with scipy` で `stats.ttest_ind` (Welch) を実行

### スクリプト確認と実行


In [ ]:
"""Independent two-sample t-test comparing two groups."""
from scipy import stats
import numpy as np

# --------------------------------------------------------------
# このスクリプトは 2 つの独立した群（旧式と新式のオンボーディングフロー）で
# 平均満足度に差があるかを Welch の t 検定で判定します。
# 帰無仮説: 両群の母平均は等しい。
# 対立仮説: 両群の母平均は異なる（両側検定）。
# Welch 法は分散が等しくない可能性がある実務データ向きです。
# --------------------------------------------------------------

# 1-7 のリッカート尺度で集計した満足度スコア（旧フロー）
legacy = np.array([4.8, 5.1, 4.5, 4.9, 5.0, 4.7, 4.6, 4.8, 5.2, 4.5])
# 新フローの満足度スコア。サンプルサイズを揃えて比較を容易にする。
new = np.array([5.4, 5.6, 5.1, 5.5, 5.7, 5.2, 5.6, 5.3, 5.7, 5.4])

# equal_var=False により Welch t 検定を選択。分散が異なる場合でも頑健。
statistic, p_value = stats.ttest_ind(new, legacy, equal_var=False)

print(f"t-statistic: {statistic:.3f}")
print(f"p-value: {p_value:.4f}")
# 解釈を簡潔に出力。p 値が 0.05 未満なら新フローが有意に高いとみなす。
print("The new onboarding flow is statistically higher" if p_value < 0.05 else "No significant difference detected.")

## 3. 対応のある t 検定
- **対象スクリプト**: `examples/t_test_paired.py`
- **目的**: 同一ドライバーの訓練前後で反応時間が短縮したかを確認します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/t_test_paired.py`

### コマンド/オプションのポイント
- `stats.ttest_rel` はペア差分を直接扱う
- `--with numpy` で Before/After 配列を整形
- データ順序を崩さないようスクリプトの配列を維持

### スクリプト確認と実行


In [ ]:
"""Paired t-test comparing before/after measurements for the same subjects."""
from scipy import stats
import numpy as np

# --------------------------------------------------------------
# 同一ドライバーのトレーニング前後の反応時間を比較し、
# トレーニングが平均反応時間を短縮したかどうかを対応のある t 検定で確認します。
# 帰無仮説: 前後の平均反応時間に差はない。
# 対立仮説: トレーニング後の反応時間は短い（ここでは両側検定で差の有無を確認）。
# --------------------------------------------------------------

# トレーニング前の反応時間（ミリ秒）。1 人あたりの値を並べる。
before = np.array([612, 598, 605, 623, 615, 607, 618, 611])
# 同じ順序でトレーニング後の反応時間を記録。位置対応が崩れないよう注意。
after = np.array([590, 582, 588, 600, 596, 589, 594, 592])

# stats.ttest_rel は各ペアの差分に基づいて t 値・p 値を算出する。
statistic, p_value = stats.ttest_rel(before, after)

print(f"t-statistic: {statistic:.3f}")
print(f"p-value: {p_value:.4f}")
# 反応時間が有意に減少したかどうかを自然言語メッセージで表示。
print("Training significantly reduced reaction time" if p_value < 0.05 else "No significant change detected.")

## 4. カイ二乗適合度検定
- **対象スクリプト**: `examples/chi_square_goodness_of_fit.py`
- **目的**: 曜日別売上が一様分布と一致するかを検定します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/chi_square_goodness_of_fit.py`

### コマンド/オプションのポイント
- `np.repeat` で一様期待度数を構築
- `stats.chisquare` に観測・期待ベクトルを渡す
- カテゴリを増やす際は `observed` を編集

### スクリプト確認と実行


In [ ]:
"""Chi-square goodness-of-fit test for checking if sales match an expected weekday distribution."""
from scipy import stats
import numpy as np

# --------------------------------------------------------------
# このスクリプトは、曜日ごとの売上分布が「全曜日で均等」という仮説と合致するかを
# カイ二乗適合度検定で調べます。
# 帰無仮説: 観測度数は一様分布（全曜日同じ割合）に従う。
# 対立仮説: 少なくとも 1 つの曜日で期待度数とずれる。
# --------------------------------------------------------------

# 実際に観測した各曜日の売上件数。1 週間分のデータを固定値で用意。
observed = np.array([52, 47, 50, 56, 65, 70, 60])
# 期待度数: 合計売上を曜日数で割った値を各曜日に割り当て、一様分布を表現。
expected = np.repeat(observed.sum() / observed.size, observed.size)

# stats.chisquare は観測度数と期待度数からカイ二乗統計量と p 値を返す。
statistic, p_value = stats.chisquare(f_obs=observed, f_exp=expected)

degrees_of_freedom = observed.size - 1
# 適合度検定ではカテゴリ数-1 が自由度になる。
print(f"Chi-square statistic: {statistic:.3f}")
print(f"Degrees of freedom: {degrees_of_freedom}")
print(f"p-value: {p_value:.4f}")
# p 値が 0.05 未満なら「均等分布からの逸脱あり」と解釈する。
print("Weekday sales deviate from uniform expectations" if p_value < 0.05 else "Sales align with a uniform weekday pattern.")

## 5. カイ二乗独立性の検定
- **対象スクリプト**: `examples/chi_square_independence.py`
- **目的**: 性別と色の嗜好が関連するかを 2×2 分割表で検定します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/chi_square_independence.py`

### コマンド/オプションのポイント
- `stats.chi2_contingency` で統計量・p 値・期待度数を取得
- `--with numpy` で分割表を `np.array` 化
- 列や行を増やす場合は行列サイズを変える

### スクリプト確認と実行


In [ ]:
"""Chi-square test of independence on a contingency table."""
from scipy import stats
import numpy as np

# --------------------------------------------------------------
# 性別（女性・男性）と、好む製品カラー（A・B）の関係を
# カイ二乗独立性の検定で確認します。
# 帰無仮説: 性別と色の嗜好は独立（関連なし）。
# 対立仮説: 性別により色の嗜好分布が異なる。
# --------------------------------------------------------------

# 2×2 の分割表。行が性別、列が色の選好を表す。サンプルは固定値。
contingency = np.array([
    [35, 22],  # 女性: カラーA / カラーB
    [28, 30],  # 男性: カラーA / カラーB
])

# stats.chi2_contingency は統計量・p 値・自由度・期待度数を一度に返す。
statistic, p_value, dof, expected = stats.chi2_contingency(contingency)

print(f"Chi-square statistic: {statistic:.3f}")
print(f"Degrees of freedom: {dof}")
print(f"p-value: {p_value:.4f}")
# 期待度数との乖離が大きければ独立性が否定され、関連があると判断。
print("Preference depends on gender" if p_value < 0.05 else "Preference independent of gender.")
# 期待度数をあわせて表示し、どのセルで差が生じたかを確認できるようにする。
print("Expected counts:\n", expected)

## 6. カイ二乗同質性の検定
- **対象スクリプト**: `examples/chi_square_homogeneity.py`
- **目的**: 複数クリエイティブ間でコンバージョン率が同じかを比較します。
- **実行コマンド**: `uv run --with numpy --with scipy python3 examples/chi_square_homogeneity.py`

### コマンド/オプションのポイント
- 行がキャンペーン、列が結果（成約/非成約）
- `stats.chi2_contingency` を再利用
- 期待度数で差の要因を把握

### スクリプト確認と実行


In [ ]:
"""Chi-square test of homogeneity comparing conversion across campaigns."""
from scipy import stats
import numpy as np

# --------------------------------------------------------------
# 3 種類の広告クリエイティブ（A/B/C）で、コンバージョン率が同じかどうかを
# カイ二乗同質性の検定で確かめます。
# 帰無仮説: すべてのクリエイティブで転換率の比率は同じ。
# 対立仮説: 少なくとも 1 つのクリエイティブで比率が異なる。
# --------------------------------------------------------------

# 行がクリエイティブ、列が「成約」「非成約」を表す 3×2 の分割表。
# キャンペーンごとの集計値を固定にすることで、結果を再現しやすくする。
contingency = np.array([
    [42, 158],
    [55, 145],
    [33, 170],
])

# chi2_contingency は同質性検定にも利用でき、期待度数も返してくれる。
statistic, p_value, dof, expected = stats.chi2_contingency(contingency)

print(f"Chi-square statistic: {statistic:.3f}")
print(f"Degrees of freedom: {dof}")
print(f"p-value: {p_value:.4f}")
# 閾値 5% で転換率の差があるかどうかを判定して表示。
print("Conversion rate differs by creative" if p_value < 0.05 else "No evidence of conversion difference among creatives.")
print("Expected counts:\n", expected)